In [33]:
print('Importing')
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import torchvision
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import numpy as np
from DenseNet_model import DenseNet
from PIL import Image
print('Done importing')

Importing
Done importing


In [22]:
pathology = 'Enlarged Cardiomediastinum'

In [23]:
hpc = False
print(sys.argv)
if (len(sys.argv) > 1 and sys.argv[1] == 'hpc'):
    hpc = True

['/Users/wilsonduan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py', '--f=/Users/wilsonduan/Library/Jupyter/runtime/kernel-v2-873UbFJ7XaRsHet.json']


In [24]:
lr = 0.0002
n_epochs = 20
batch_size = 128
device = torch.device('cuda' if (torch.cuda.is_available()) else 'cpu')
print(hpc, device, n_epochs)

False cpu 20


In [25]:
if (hpc):
    labels_path_train = '/groups/CS156b/data/student_labels/train2023.csv'
    labels_path_test = '/groups/CS156b/data/student_labels/test_ids.csv'
    img_dir = '/groups/CS156b/data'

    df_train = pd.read_csv(labels_path_train)[:-1]
else:
    labels_path_train = 'data/train/labels/labels.csv'
    labels_path_test = 'data/test/ids.csv'
    img_dir = 'data'

    df_train = pd.read_csv(labels_path_train)

df_train = df_train[['Path', pathology]]
df_train = df_train.dropna()
df_test = pd.read_csv(labels_path_test)
print(df_train.head())
print(df_test.head())

                                        Path  Enlarged Cardiomediastinum
5    train/pid17532/study1/view1_frontal.jpg                        -1.0
6    train/pid17532/study1/view2_lateral.jpg                        -1.0
7    train/pid05208/study1/view1_frontal.jpg                        -1.0
9    train/pid35409/study2/view1_frontal.jpg                         1.0
14  train/pid18261/study13/view1_frontal.jpg                        -1.0
   Unnamed: 0  Id                                    Path
0           0  18  test/pid56785/study1/view1_frontal.jpg
1           1  19  test/pid56785/study1/view2_lateral.jpg
2           2  44  test/pid57943/study1/view1_frontal.jpg
3           3  45  test/pid57943/study2/view1_frontal.jpg
4           4  57  test/pid54805/study1/view1_frontal.jpg


In [26]:
def parse_labels(df):
    df = df[['Path', pathology]]
    df = df.dropna()
    return df

class TrainImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        if (hpc):
            self.img_labels = parse_labels(pd.read_csv(annotations_file)[:-1])
        else:
            self.img_labels = parse_labels(pd.read_csv(annotations_file))
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        row = self.img_labels.iloc[idx]

        img_path = row['Path']
        img_path = os.path.join(self.img_dir, img_path)

        image = Image.open(img_path) # PIL image for applying transform for pre-trained ResNet model 
        label_num = row[-1] + 1 # -1 => 0, 0 => 1, 1 => 2
        label = torch.tensor(label_num).long()

        if self.transform:
            image = self.transform(image)

        return image, label
    
class TestImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        row = self.img_labels.iloc[idx]

        img_path = row['Path']
        img_path = os.path.join(self.img_dir, img_path)

        # image = read_image(img_path)
        image = Image.open(img_path) # PIL image for applying transform for pre-trained ResNet model 
        label = row['Id']

        if self.transform:
            image = self.transform(image)

        return image, label

In [27]:
# default transform:
transform = transforms.Compose([
    transforms.Lambda(lambda image: image.convert('RGB')),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# transform with random flipping and cropping:
# transform = transforms.Compose([
#     transforms.Lambda(lambda image: image.convert('RGB')),
#     transforms.Resize((300, 300)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomCrop((256, 256))
# ])

# transform with Gaussian blur:
# transform = transforms.Compose([
#     transforms.Lambda(lambda image: image.convert('RGB')),
#     transforms.Resize((256, 256)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
#     transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0))
# ])

training_data = TrainImageDataset(labels_path_train, img_dir, transform=transform)
test_data = TestImageDataset(labels_path_test, img_dir, transform=transform)

train_size = int(0.8 * len(training_data))
val_size = len(training_data) - train_size
training_data, val_data = torch.utils.data.random_split(training_data, [train_size, val_size])

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [30]:
model = DenseNet(channels=3, growth_rate=16, num_classes=3)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.5, 0.999))

In [29]:
# store metrics
training_loss_history = np.zeros(n_epochs)
validation_loss_history = np.zeros(n_epochs)

for epoch in range(n_epochs):
    print(f'Epoch {epoch+1}/{n_epochs}:')
    # train
    model.train()
    for i, data in enumerate(train_dataloader):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        # forward pass
        output = model(images)
        # calculate categorical cross entropy loss
        print('output', output.shape)
        print('label', labels.shape)
        loss = criterion(output, labels)
        # backward pass
        loss.backward()
        optimizer.step()

        # track training loss
        training_loss_history[epoch] += loss.item()
    
    training_loss_history[epoch] /= len(train_dataloader)
    print(f'Training Loss: {training_loss_history[epoch]:0.4f}')

    # validate
    with torch.no_grad():
        model.eval()
        for i, data in enumerate(val_dataloader):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            # forward pass
            output = model(images)
            # find loss
            loss = criterion(output, labels)
            validation_loss_history[epoch] += loss.item()
        validation_loss_history[epoch] /= len(val_dataloader)
    print(f'Validation loss: {validation_loss_history[epoch]:0.4f}')

Epoch 1/20:


/Users/wilsonduan/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459064158/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


output torch.Size([5, 3])
label torch.Size([5])
Training Loss: 1.0962
Validation loss: 1.0336
Epoch 2/20:
output torch.Size([5, 3])
label torch.Size([5])
Training Loss: 0.9769
Validation loss: 0.9794
Epoch 3/20:
output torch.Size([5, 3])
label torch.Size([5])
Training Loss: 0.8731
Validation loss: 0.9748
Epoch 4/20:
output torch.Size([5, 3])
label torch.Size([5])
Training Loss: 0.8031
Validation loss: 0.9836
Epoch 5/20:
output torch.Size([5, 3])
label torch.Size([5])


KeyboardInterrupt: 

In [13]:
# get predictions on test set
rows_list = []
with torch.no_grad():
    model.eval()
    for i, data in enumerate(test_dataloader):
        images, ids = data
        images, ids = images.to(device), ids.to(device)
        
        output = np.array(model(images).cpu())
        output = np.argmax(output, axis=1) - 1
        for preds, id in zip(output, ids):
            rows_list.append([int(id)] + [preds])

df_output = pd.DataFrame(rows_list, columns=['Id', pathology])
df_output.head()

,Id,Enlarged Cardiomediastinum
0,18,1
1,19,1
2,44,1
3,45,1
4,57,1


In [16]:
if (hpc):
    output_dir = '/groups/CS156b/2024/BroadBahnMi/predictions'
else:
    output_dir = 'predictions'

filename = '_'.join(pathology.split())
full_path = os.path.join(output_dir, f'preds_{filename}.csv')
df_output.to_csv(full_path, index=False)